CHANGE LATER:
- file paths

# Import & Settings

In [ ]:
import solara
import solara.lab
import json
import io
from ipyleaflet import Map, basemaps, GeoJSON, LayerGroup, DrawControl, LayersControl, WidgetControl #, Rectangle, CircleMarker, WidgetControl, Polyline, AwesomeIcon, Marker, Icon
import ipywidgets as widgets

import os 
from hydromt_fiat.fiat import FIATModel
from hydromt.data_catalog.data_catalog import DataCatalog
from pathlib import Path
import geopandas as gpd
from shapely.geometry import shape
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import shutil
from hydromt_fiat.data import fetch_data

m = Map(center=(53.5, -0.5), zoom=8, scroll_wheel_zoom=True,basemap=basemaps.OpenStreetMap.Mapnik)

tab_layers = {
    'Config': LayerGroup(),
    'Prop': LayerGroup(),
    'InitBuild': LayerGroup(),
    'InspWrite': LayerGroup(),
}
current_layer_group = solara.reactive(tab_layers['Config'])
tab_controls = solara.reactive({})
# control_update_signal = solara.reactive(0)
# current_control = solara.reactive(None)
# rectangle = solara.reactive(None)

control_signals = {
    'Config': solara.reactive(0),
    'Prop': solara.reactive(0),
    'InitBuild': solara.reactive(0),
    'InspWrite': solara.reactive(0),
}

configdone = solara.reactive(False) 
buildmoddone = solara.reactive(False)


# Tabs

## 1) Configuration, Initialisation & model setup

In [ ]:
model_name = solara.reactive("Humber")
continuous_update = solara.reactive(True)

model_root = solara.reactive("C:\\Users\\santjer\\OneDrive - Stichting Deltares\\Documents\\IRISCC\\FIAT\\model")# ("C:\\") # CHANGE PATH HERE


draw_control = DrawControl(rectangle={"shapeOptions": {"color": "#0000FF"}},polygon={},circle={},polyline={},circlemarker={})
draw_control.edit = False

button_fix_rect_enabled = solara.reactive(False)
draw_message = solara.reactive("")
def handle_draw(target, action, geo_json):
    button_fix_rect_enabled.set(len(draw_control.data) == 0)
    if action == "deleted":
        if len(draw_control.data) <= 2: # 2 because it is AT THE TIME OF THE CALL, not what the len is afterwards!
            draw_message.set("")
        else:
            draw_message.set("You drew too many rectangles. Only place one.")
    elif len(draw_control.data) == 0:
        draw_message.set("")
    else:
        draw_message.set("You drew too many rectangles. Only place one.")
draw_control.on_draw(handle_draw)


set_model_status = solara.reactive("Idle")
def set_model():
    global model_folder
    model_folder = Path(model_root.value) / model_name.value  # full path to model folder
    data_catalog = (Path(os.path.abspath("")) / "Data" / "data_catalog.yml")  # path to data catalog relative to this notebook
    set_model_status.set("Done")


chosen_rectangle = solara.reactive("No rectangle set-up yet.")
def fix_model_domain():
    global mod_dom_rect, region
    mod_dom_rect = draw_control.data[0] # last_draw # last_draw has the problem problem that, if it is deleted, it is still saved as last_draw
    if mod_dom_rect and "geometry" in draw_control.last_draw:
        coords = draw_control.last_draw['geometry']['coordinates'][0][0:4]
        chosen_rectangle.set(f"Chosen rectangle between (lon, lat): ({coords[0][0]:.2f}, {coords[0][1]:.2f}) and ({coords[2][0]:.2f}, {coords[2][1]:.2f})")
    else:
        chosen_rectangle.set("No rectangle set-up yet.")

    region = gpd.GeoDataFrame(geometry=[shape(mod_dom_rect['geometry'])], crs="EPSG:4326")

    configdone.set(True) 




# Europe, North America, Central America, South America, Asia, Africa, Oceania, or Global
continent_options = ["asia", "africa", "north america", "central and south america", "oceania", "europe", "global (not fully implemented yet)"]
selected_continent = solara.reactive("") # Europe

country_options = solara.reactive([])
continent_to_countries = {
    "asia": ["Bangladesh", "Cambodia", "China", "India", "Indonesia", "Japan", "Laos", "Pakistan", "Philippines", "Taiwan", "Thailand", "Vietnam"],
    "africa": ["Malawi" , "Mozambique", "Nigeria", "South-Africa"],
    "north america": ["USA", "Canada"],
    "central and south america": ["Argentina", "Bolivia", "Brazil", "Colombia", "El Salvador", "Guatemala", "Haiti", "Mexico", "St. Maarten", "continental"],
    "oceania": ["Australia", "New Zealand"],
    "europe": ["Belgium", "Czech Republic", "Denmark", "France", "Germany", "Hungary", "Norway", "Sweden", "Switzerland", "The Netherlands", "United Kingdom"],
    "global (not fully implemented yet)": []
}
selected_country = solara.reactive("") # United Kingdom


In [ ]:
@solara.component
def Tab_Configuration():

    tab_controls.value["Config"] = draw_control
    solara.use_effect(lambda: country_options.set(continent_to_countries.get(selected_continent.value, [])), [selected_continent.value])
    
    with solara.Card("Configuration", style={"width": "100%", "padding": "10px"}):
        solara.Markdown("**Select Model Name:**")
        solara.InputText("Model Name (avoid spaces)", value=model_name, continuous_update=continuous_update.value, style={"marginBottom": "20px"})

        solara.Markdown("**Select Model Path:**")
        solara.InputText("Model path", value=model_root, continuous_update=True, style={"marginBottom": "20px"})
        path_model = Path(model_root.value)
        if model_root.value:
            if not path_model.exists():
                solara.Error("Path does not exist.")
        
        solara.Button("Set Path",on_click=set_model, continuous_update=True, style={"marginBottom": "20px"})
        if set_model_status.value == "Done":
            solara.Markdown("**Done!**")

        solara.Markdown("**Model Domain:**")
        solara.Markdown("Draw Model Domain as one rectangle in map.")
        if draw_message.value:
            solara.Error(f"{draw_message.value}")
        else:
            solara.Markdown(" ")
        solara.Button("Fix Model Domain",on_click=fix_model_domain, continuous_update=True, disabled=not button_fix_rect_enabled.value, style={"marginBottom": "20px"})
        solara.Markdown(chosen_rectangle.value)#f"Chosen rectangle: ({draw_control.last_draw['geometry']['coordinates'][0][0:4][0][0]:.2f}, {draw_control.last_draw['geometry']['coordinates'][0][0:4][0][1]:.2f}) and ({draw_control.last_draw['geometry']['coordinates'][0][0:4][2][0]:.2f}, {draw_control.last_draw['geometry']['coordinates'][0][0:4][2][1]:.2f})")

        solara.Markdown("**Select continent and country of the domain:**")
        solara.Select(label="Continent", value=selected_continent, values=continent_options)
        solara.Select(label="Select Country", value=selected_country, values=country_options.value)
        solara.Markdown(f"Selected country: **{selected_country.value}**")

    with solara.Row(justify="end"):     
        solara.Button(label="Go to Step 2.", on_click=lambda: selected_tab.set('Prop'),disabled=not configdone.value)


# Tab_Configuration()

## 2) Vulnerabilty, Exposure and Max. damage

In [ ]:
# Vulnerability <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
# vulnerability_fname: Path to vulnerability dataset file or an entry in the data catalog that points to the vulnerability dataset file.
vul_path_or_catalog = solara.reactive("Data catalog entry") #; vul_catalog_option = solara.reactive("default")
vulnerability_fname_inp = solara.reactive("vulnerability_curves") 
vulnerability_fname = solara.reactive("")

# vulnerability_linking_fname: Path or data catalog entry of the vulnerability linking table. 
# If not provided, it is assumed that the ‘type’ in the vulnerability dataset is correct, by default None. # <<< IGNORED
vul_linking_path_or_catalog = solara.reactive("Data catalog entry")
vulnerability_linking_fname_inp = solara.reactive("vulnerability_curves_linking")
vulnerability_linking_fname = solara.reactive("")

# unit: The unit which the vulnerability index is in, by default “m” for meters.
unit = solara.reactive("m")

# Exposure <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
# exposure_fname: The name of or path to the raw exposure dataset.
exp_path_or_catalog = solara.reactive("Data catalog entry")
exposure_fname_inp = solara.reactive("osm_buildings")
exposure_fname = solara.reactive("")

# exposure_type_column: The name of column in the raw dataset that specifies the object type, e.g. the occupancy type.
exp_type_option = solara.reactive("default")
exposure_type_column = solara.reactive("building")

# exposure_link_fname: The name of or path to the dataset containing the mapping of the exposure types to
# the vulnerability data, by default None.
exp_link_option = solara.reactive("Name of dataset")
exposure_link_fname_inp = solara.reactive("osm_buildings_link") 
exposure_link_fname = solara.reactive("")

# Max. damage <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
# exposure_name: The name of the existing dataset.
exposure_name = solara.reactive("osm_buildings")

# exposure_type: Type of exposure corresponding with the vulnerability data, e.g. ‘damage’.
exposure_type = solara.reactive("damage")

# exposure_cost_table_fname: The name of/ path to the mapping of the costs per subtype of the exposure type, 
# e.g. ‘residential_structure’ or ‘residential_content’.
exp_cost_option = solara.reactive("Name of mapping of the costs")
exposure_cost_table_fname_inp = solara.reactive("damage_values")
exposure_cost_table_fname = solara.reactive("")

# # # # # # # # # # # # # # # # exposure_cost_link_fname: A linking table to like the present object type with the identifiers defined in the cost table.
# # # # # # # # # # # # # # # # If None, it is assumed the present object type matches the identifiers in the cost table. By default None.

# ground_flht: the height of the ground floor of all assets, in the same unit. It can also be an array 
# if assets have different ground floor heights
ground_flth = solara.reactive(0.0)
ground_elevatn = solara.reactive(0.0)
extract_method = solara.reactive("centroid")


In [ ]:
@solara.component
def Tab_Properties():
    with solara.lab.Tabs(): 
        with solara.lab.Tab("Vulnerability"):
            with solara.Card("Setup Vulnerability Parameters", style={"width": "100%", "padding": "10px"}):
                # either path (input text) or 
                # entry in data catalog pointing to vulnerability dataset file --> either default or custom (dann input text)
                # A: input text
                # B: either default or custom --> if default: see vscode inputs OR if custom: input text
                solara.Markdown("1.) Vulnerability dataset", style={"fontSize": "24px"})
                solara.Text("Choose path to vulnerability dataset or data catalog entry:")
                solara.Select(label="Choose", values=["Path to vulnerability dataset", "Data catalog entry"], value=vul_path_or_catalog)
                if vul_path_or_catalog.value == "Path to vulnerability dataset":
                    solara.InputText(label="Enter Path", value=vulnerability_fname_inp, style={"marginBottom": "20px"})
                    vulnerability_fname.set(Path(vulnerability_fname_inp.value))
                    solara.Markdown(f"Chosen path: {vulnerability_fname.value}")
                    if not vulnerability_fname.value.exists():
                        solara.Error("File/Path does not exist.")
                elif vul_path_or_catalog.value == "Data catalog entry":
                    solara.Text("Data catalog entry:")
                    solara.InputText(label="Enter data catalog entry", value=vulnerability_fname_inp, style={"marginBottom": "20px"})
                    vulnerability_fname.set(vulnerability_fname_inp.value)
                    solara.Markdown(f"Chosen entry: {vulnerability_fname.value}")

                solara.Markdown("2.) Vulnerability linking", style={"fontSize": "24px"})
                solara.Text("Choose path to vulnerability linking table or data catalog entry:")
                solara.Select(label="Choose", values=["Path to vulnerability linking dataset", "Data catalog entry"], value=vul_linking_path_or_catalog)
                if vul_linking_path_or_catalog.value == "Path to vulnerability linking dataset":
                    solara.InputText(label="Enter Path", value=vulnerability_linking_fname_inp, style={"marginBottom": "20px"})
                    vulnerability_linking_fname.set(Path(vulnerability_linking_fname_inp.value))
                    solara.Markdown(f"Chosen path: {vulnerability_linking_fname.value}")
                    if not vulnerability_linking_fname.value.exists():
                        solara.Error("File/Path does not exist.")
                elif vul_linking_path_or_catalog.value == "Data catalog entry":
                    solara.Text("Data catalog entry:")
                    solara.InputText(label="Enter data catalog entry", value=vulnerability_linking_fname_inp, style={"marginBottom": "20px"})
                    vulnerability_linking_fname.set(vulnerability_linking_fname_inp.value)
                    solara.Markdown(f"Chosen entry: {vulnerability_linking_fname.value}")

                solara.Markdown("3.) Unit", style={"fontSize": "24px"})
                solara.Text("Choose the unit in which the vulnerability index is in.")
                solara.Select(label="Unit", values=["m"], value=unit.value)

                with solara.Row(justify="end"):  
                    solara.Button(label="Go to Step 3.", on_click=lambda: selected_tab.set('InitBuild'))

        
        with solara.lab.Tab("Exposure"):
            with solara.Card("Setup Exposure Parameters", style={"width": "100%", "padding": "10px"}):
                
                solara.Markdown("1.) Exposure dataset", style={"fontSize": "24px"})
                solara.Text("Choose path to exposure dataset or data catalog entry:")
                solara.Select(label="Choose", values=["Path to exposure dataset", "Data catalog entry"], value=exp_path_or_catalog)
                if exp_path_or_catalog.value == "Path to exposure dataset":
                    solara.InputText(label="Enter Path", value=exposure_fname_inp, style={"marginBottom": "20px"})
                    exposure_fname.set(Path(exposure_fname_inp.value))
                    solara.Markdown(f"Chosen path: {exposure_fname.value}")
                    if not exposure_fname.value.exists():
                        solara.Error("File/Path does not exist.")
                elif exp_path_or_catalog.value == "Data catalog entry":
                    solara.Text("Data catalog entry:")
                    solara.InputText(label="Enter data catalog entry", value=exposure_fname_inp, style={"marginBottom": "20px"})
                    exposure_fname.set(exposure_fname_inp.value)
                    solara.Markdown(f"Chosen entry: {exposure_fname.value}")
                    

                solara.Markdown("2.) Exposure type", style={"fontSize": "24px"})
                solara.Text("Choose name of the column in the raw dataset, that specifies the object type.")
                solara.Select(label="Choose", values=["default", "custom"], value=exp_type_option)
                if exp_type_option.value == "default":
                    exposure_type_column.set("building")
                elif exp_type_option.value == "custom":
                    solara.InputText(label="Enter data catalog entry", value=exposure_type_column, style={"marginBottom": "20px"})

                solara.Markdown("3.) Exposure dataset containing the mapping of the exposure types", style={"fontSize": "24px"})
                solara.Text("Choose name of or path to the dataset.")
                solara.Select(label="Choose", values=["Name of dataset", "Path to the dataset"], value=exp_link_option)
                if exp_link_option.value == "Path to the dataset":
                    solara.InputText(label="Enter Path", value=exposure_link_fname_inp, style={"marginBottom": "20px"})
                    exposure_link_fname.set(Path(exposure_link_fname_inp.value))
                    solara.Markdown(f"Chosen path: {exposure_link_fname.value}")
                    if not exposure_link_fname.value.exists():
                        solara.Error("File/Path does not exist.")
                elif exp_link_option.value == "Name of dataset":
                    solara.Text("Name of dataset:")
                    solara.InputText(label="Enter name of dataset", value=exposure_link_fname_inp, style={"marginBottom": "20px"})
                    exposure_link_fname.set(exposure_link_fname_inp.value)
                    solara.Markdown(f"Chosen entry: {exposure_link_fname.value}")

                with solara.Row(justify="end"):  
                    solara.Button(label="Go to Step 3.", on_click=lambda: selected_tab.set('InitBuild'))
                
                
        with solara.lab.Tab("Maximum damage"):
            with solara.Card("Maximum damage", style={"width": "100%", "padding": "10px"}):
                
                solara.Markdown("1.) Exposure dataset name", style={"fontSize": "24px"})
                solara.Select(label="Choose", values=["osm_buildings"], value=exposure_name) # OR INPUTTEXT

                solara.Markdown("2.) Type of Exposure corresponding with the Vulnerability data", style={"fontSize": "24px"})
                solara.Select(label="Choose", values=["damage"], value=exposure_type)

                solara.Markdown("3.) Costs per subtype of exposire type", style={"fontSize": "24px"})
                solara.Text("Choose name of or path to the dataset.")
                solara.Select(label="Choose", values=["Name of mapping of the costs", "Path to the mapping of the costs"], value=exp_cost_option)
                if exp_cost_option.value == "Path to the mapping of the costs":
                    solara.InputText(label="Enter Path", value=exposure_cost_table_fname_inp, style={"marginBottom": "20px"})
                    exposure_cost_table_fname.set(Path(exposure_cost_table_fname_inp.value))
                    solara.Markdown(f"Chosen path: {exposure_cost_table_fname.value}")
                    if not exposure_cost_table_fname.value.exists():
                        solara.Error("File/Path does not exist.")
                elif exp_cost_option.value == "Name of mapping of the costs":
                    solara.Text("Name of dataset:")
                    solara.InputText(label="Enter name of dataset", value=exposure_cost_table_fname_inp, style={"marginBottom": "20px"})
                    exposure_cost_table_fname.set(exposure_cost_table_fname_inp.value)
                    solara.Markdown(f"Chosen entry: {exposure_cost_table_fname.value}")

                # # THIS IS NOT IN THE NOTEBOOK!
                # solara.Markdown("4.) Linked Table, objects and costs", style={"fontSize": "24px"})
                # # exposure_cost_link_fname
                solara.Markdown("4.) Heights of gound floor and elevation", style={"fontSize": "24px"})
                solara.InputFloat("Enter the height of the ground floor of all assets: ", value=ground_flth, continuous_update=True, style={"marginBottom": "20px"})
                solara.InputFloat("Enter the height of the ground elevation: ", value=ground_elevatn, continuous_update=True, style={"marginBottom": "20px"})
                solara.Select(label="Choose extract method: ", values=["centroid"], value=extract_method)

                with solara.Row(justify="end"):  
                    solara.Button(label="Go to Step 3.", on_click=lambda: selected_tab.set('InitBuild'))

# Tab_Properties()

## 3) Initialising & Build Model

In [ ]:
# # # WITHOUT SOLARA SUCCESSFUL!!! <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
# # model_name = 'Humber'
# # # model_root = 'FIAT'
# # model_root = r'C:\Users\santjer\OneDrive - Stichting Deltares\Documents\IRISCC\FIAT\model'

# # model_folder = Path(model_root) / model_name  # full path to model folder
# # data_catalog = (
# #     Path(os.path.abspath("")) / "Data" / "data_catalog.yml"
# # )  # path to data catalog relative to this notebook

# # region_fn = "Data/map.geojson"
# # region = gpd.read_file(Path(model_root) / region_fn)
# # region.explore()

# # continent = 'europe' 
# # country = 'United Kingdom' 


# # vulnerability_fname = "vulnerability_curves" # to download OSM data
# # vulnerability_linking_fname = "vulnerability_curves_linking" # to be used with OSM data
# # unit = "m" # meters
# # exposure_fname="osm_buildings" # to use OSM data
# # exposure_type_column="building" #  specific for OSM data
# # exposure_link_fname="osm_buildings_link"
# # exposure_name="osm_buildings"
# # exposure_type="damage"
# # exposure_cost_table_fname="damage_values"
# # ground_flht = 0
# # ground_elevatn = 0
# # extract_method = "centroid"


# # if model_folder.exists():
# #     shutil.rmtree(model_folder)
# # build_data = fetch_data(data="build-data")
# # fiat = FIATModel(root=model_root, data_libs=Path(build_data,"data_catalog.yml"), mode='w+')
# # fiat.setup_region(region=region)
# # fiat.vulnerability.setup(vulnerability_fname=vulnerability_fname,
# #                          vulnerability_linking_fname=vulnerability_linking_fname,
# #                          unit=unit,
# #                          continent=continent
# #                          )
# # fiat.exposure_geoms.setup(exposure_fname=exposure_fname,
# #                           exposure_type_column=exposure_type_column,
# #                           exposure_link_fname=exposure_link_fname
# #                           ) 


# ################################################################################################################################
# ################################################################################################################################
# ################################################################################################################################
# ################################################################################################################################
# ################################################################################################################################


# # WITH SOLARA NOT SUCCESSFUL!!! <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
# model_name = solara.reactive("Humber")
# model_root = solara.reactive("C:\\Users\\santjer\\OneDrive - Stichting Deltares\\Documents\\IRISCC\\FIAT\\model")
# model_folder = Path(model_root.value) / model_name.value 

# # here drawing of region rectangle
# region = gpd.read_file(Path(model_root.value) / "Data/map.geojson")
# # region.explore()

# selected_continent = solara.reactive("europe") # Europe
# selected_country = solara.reactive("United Kingdom") # United Kingdom


# # Vulnerability <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
# vulnerability_fname = solara.reactive("vulnerability_curves") 
# vulnerability_linking_fname = solara.reactive("vulnerability_curves_linking")
# unit = solara.reactive("m")
# exposure_fname = solara.reactive("osm_buildings")
# exposure_type_column = solara.reactive("building")
# exposure_link_fname = solara.reactive("osm_buildings_link") 
# exposure_name = solara.reactive("osm_buildings")
# exposure_type = solara.reactive("damage")
# exposure_cost_table_fname = solara.reactive("damage_values")
# ground_flth = solara.reactive(0.0)
# ground_elevatn = solara.reactive(0.0)
# extract_method = solara.reactive("centroid")


# if model_folder.exists():
#     shutil.rmtree(model_folder)
# build_data = fetch_data(data="build-data")
# fiat = FIATModel(root=model_root.value, data_libs=Path(build_data,"data_catalog.yml"), mode='w+')
# # fiat = FIATModel(root=model_root, data_libs=Path(build_data,"data_catalog.yml"), mode='w+')


# fiat.setup_region(region=region)

# print(vulnerability_fname.value == 'vulnerability_curves')
# print(vulnerability_linking_fname.value == 'vulnerability_curves_linking')
# print(unit.value == 'm')
# print(selected_continent.value == 'europe')
# fiat.vulnerability.setup(vulnerability_fname=vulnerability_fname.value,
#                          vulnerability_linking_fname=vulnerability_linking_fname.value,
#                          unit=unit.value,
#                          continent=selected_continent.value #continent#
#                          )

# print(exposure_fname.value == 'osm_buildings')
# print(exposure_type_column.value == 'building')
# print(exposure_link_fname.value == 'osm_buildings_link')

# fiat.exposure_geoms.setup(exposure_fname=exposure_fname.value,
#                           exposure_type_column=exposure_type_column.value,
#                           exposure_link_fname=exposure_link_fname.value
#                           ) 

# # fiat.exposure_geoms.setup_max_damage(exposure_name=exposure_name,
# #                                      exposure_type=exposure_type,
# #                                      exposure_cost_table_fname=exposure_cost_table_fname,
# #                                      country=country.value
# #                                      )


In [ ]:
# further notes
# if model_folder.exists():
#     shutil.rmtree(model_folder)
# build_data = fetch_data(data="build-data")
# fiat = FIATModel(root=model_root.value, data_libs=Path(build_data,"data_catalog.yml"), mode='w+')

# fiat.setup_region(region=region)
# fiat.vulnerability.setup(vulnerability_fname=vulnerability_fname.value,
#                          vulnerability_linking_fname=vulnerability_linking_fname.value,
#                          unit=unit.value,
#                          continent=selected_continent.value
#                          )
# fiat.exposure_geoms.setup(exposure_fname=exposure_fname.value,
#                           exposure_type_column=exposure_type_column.value,
#                           exposure_link_fname=exposure_link_fname.value
#                           ) 

In [ ]:
# implement Step 2 & 3

init_model_status = solara.reactive(False)
init_model_text_status = solara.reactive("Idle")
def init_model():
    global fiat
    init_model_text_status.set("Running")
    if model_folder.exists():
        shutil.rmtree(model_folder)
    build_data = fetch_data(data="build-data")
    fiat = FIATModel(root=model_root.value, data_libs=Path(build_data,"data_catalog.yml"), mode='w+')
    init_model_status.set(True)
    init_model_text_status.set("Done")


build_model_status = solara.reactive("Idle")
def build_model():
    build_model_status.set("Running")
    fiat.setup_region(region=region)
    fiat.vulnerability.setup(vulnerability_fname=vulnerability_fname.value,
                             vulnerability_linking_fname=vulnerability_linking_fname.value,
                             unit=unit.value,
                             continent=selected_continent.value
                             )
    fiat.exposure_geoms.setup(exposure_fname=exposure_fname.value,
                          exposure_type_column=exposure_type_column.value,
                          exposure_link_fname=exposure_link_fname.value
                          ) 
    fiat.exposure_geoms.setup_max_damage(exposure_name=exposure_name.value,
                                     exposure_type=exposure_type.value,
                                     exposure_cost_table_fname=exposure_cost_table_fname.value,
                                     country=selected_country.value
                                     )
    build_model_status.set("Done")
    buildmoddone.set(True) 



In [ ]:
@solara.component
def Tab_Init_Build():
    solara.Button("Initialise Model",on_click=init_model, continuous_update=True, style={"marginBottom": "20px"})
    if init_model_text_status.value == "Running":
        solara.Markdown("Model is being initalised... Please wait.")
    elif init_model_text_status.value == "Done":
        solara.Markdown("**Model is initialised!**")
    else:
        solara.Markdown("")
    
    solara.Button("Build Model",on_click=build_model, continuous_update=True, disabled=not init_model_status.value, style={"marginBottom": "20px"})
    if build_model_status.value == "Running":
        solara.Markdown("Model is being built... Please wait. This might take a couple of minutes.")
    elif build_model_status.value == "Done":
        solara.Markdown("**Done, model is built.**")
    else:
        solara.Markdown("")

    with solara.Row(justify="end"):     
        solara.Button(label="Go to Step 4.", on_click=lambda: selected_tab.set('InspWrite'),disabled=not buildmoddone.value)
    

# Tab_Init_Build()

## 4) Inspect and write Model

In [ ]:
insp_model_status = solara.reactive("Idle")
def inspect_model():
    insp_model_status.set("Running")
    def gdf_to_geojson_layer(gdf, name=None, style=None, style_callback=None):
        data = json.loads(gdf.to_json())
        kwargs = {"data": data,"name": name or "Layer"}
        if style is not None:
            kwargs["style"] = style
        if callable(style_callback):
            kwargs["style_callback"] = style_callback
        return GeoJSON(**kwargs)
 
    def style_by_object_type(feature):
        global color_map
        obj = feature["properties"].get("object_type")
        color_map = {"residential": "#9EDAE5",
            "commercial": "#1F77B4",
            "industrial": "#8C564B"}
        color = color_map.get(obj, "#8c564b")
        return {"color": color,"fillColor": color,"fillOpacity": 0.2,"weight": 1}

    gdf = fiat.exposure_geoms.data["osm_buildings"]
    region_layer = gdf_to_geojson_layer(region, name="Region",style={"color": "black", "fillColor": "transparent", "fillOpacity": 0.0, "weight": 1})
    buildings_layer = gdf_to_geojson_layer(gdf, name="object_type", style_callback=style_by_object_type)
    
    if region_layer in current_layer_group.value.layers:
        current_layer_group.value.remove_layer(region_layer)
    current_layer_group.value.add_layer(region_layer)
    if buildings_layer in current_layer_group.value.layers:
        current_layer_group.value.remove_layer(buildings_layer)
    current_layer_group.value.add_layer(buildings_layer)

    # LEGEND
    legend_items = []
    for label, color in color_map.items():
        item = widgets.HBox([widgets.HTML(value=f"""<div style="width:14px;height:14px;background:{color};border:1px solid #333;margin-right:6px;"></div>"""),widgets.Label(label),])
        legend_items.append(item)
    legend_box = widgets.VBox([widgets.HTML("<b>object_type</b>"),*legend_items])
    legend_control = WidgetControl(widget=legend_box, position="bottomright")
    tab_controls.set({**tab_controls.value, "InspWrite": legend_control}) # tab_controls.value["InspWrite"] = legend_control
    control_signals["InspWrite"].set(control_signals["InspWrite"].value + 1) 

    insp_model_status.set("Done")
    


write_model_status = solara.reactive("Idle")
def write_model():
    write_model_status.set("Running")
    fiat.write()
    write_model_status.set("Done")


In [ ]:
@solara.component
def Tab_Insp_Write():
    solara.Button("Inspect Model",on_click=inspect_model, continuous_update=True, style={"marginBottom": "20px"})
    if insp_model_status.value == "Running":
        solara.Markdown("Model is being inspected... Please wait.")
    elif insp_model_status.value == "Done":
        solara.Markdown("**Done!**")
    else:
        solara.Markdown("")

    solara.Button("Write Model",on_click=write_model, continuous_update=True, style={"marginBottom": "20px"})
    if write_model_status.value == "Running":
        solara.Markdown("Model is being written... Please wait.")
    elif write_model_status.value == "Done":
        solara.Markdown("**Model is written!**")
    else:
        solara.Markdown("")


# Tab_Insp_Write()

# Final

In [ ]:
def lighten_color(hex_color, amount=0.5):
    hex_color = hex_color.lstrip('#')
    r, g, b = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
    r, g, b = [int(x + (255 - x) * amount) for x in (r, g, b)]
    return f'#{r:02x}{g:02x}{b:02x}'
selected_tab = solara.reactive('Config') 
current_control = solara.reactive(None)

@solara.component
def SettingsTabs():
    solara.use_effect(lambda: selected_tab.set(selected_tab.value), [selected_tab.value])

    def update_map_layers_and_controls():
        for layer in list(m.layers[1:]):
            m.remove_layer(layer)
        current_layer_group.set(tab_layers[selected_tab.value])
        m.add_layer(current_layer_group.value)
        # controls
        if current_control.value is not None and current_control.value in m.controls:
            m.remove_control(current_control.value)

        # NEU NEU NEU NEU NEU NEU NEU NEU NEU NEU NEU NEU
        for control in list(m.controls):
            if isinstance(control, DrawControl):
                m.remove_control(control)
        #################################################
        
        if selected_tab.value in tab_controls.value:
            control = tab_controls.value[selected_tab.value]
            if control not in m.controls:
                m.add_control(control)
            current_control.set(control)
        else:
            current_control.set(None)
    # solara.use_effect(update_map_layers_and_controls, [selected_tab.value, control_signals[selected_tab.value].value]) 
    solara.use_effect(update_map_layers_and_controls, [selected_tab.value, control_signals[selected_tab.value].value, tab_controls.value]) 

    def get_button_style(tab_name, original_color, font_color):
        if selected_tab.value == tab_name:
            return {"background-color": lighten_color(original_color), "color": font_color}
        return {"background-color": original_color, "color": font_color}
        

    with solara.Column(style={"width": "100%", "align-items": "center"}):
        with solara.Sidebar():
            with solara.Column(gap="10px", style={"align-items": "stretch"}):
                solara.Button("1. Configuration", on_click=lambda: selected_tab.set('Config'), 
                              style=get_button_style('Config', "#999999","black"))
                solara.Button("2. Specification of Properties", on_click=lambda: selected_tab.set('Prop'), 
                              style=get_button_style('Prop', "#999999","black"), disabled=not configdone.value)
                if not configdone.value: 
                    solara.Markdown("Model needs to be configurated first.") 

                solara.Button("3. Initialise and build Model", on_click=lambda: selected_tab.set('InitBuild'), 
                              style=get_button_style('InitBuild', "#999999","black"), disabled=not configdone.value)
                if not configdone.value: 
                    solara.Markdown("Model needs to be configurated first.") 

                solara.Button("4. Inspect and Write Model", on_click=lambda: selected_tab.set('InspWrite'), 
                              style=get_button_style('InspWrite', "#999999","black"), disabled=not buildmoddone.value)
                if not buildmoddone.value:
                    solara.Markdown("Initialise and build Model first.") 




    if selected_tab.value == 'Config':
        Tab_Configuration()
    elif selected_tab.value == 'Prop':
        Tab_Properties()
    elif selected_tab.value == 'InitBuild':
        Tab_Init_Build()
    elif selected_tab.value == 'InspWrite':
        Tab_Insp_Write()

    

@solara.component
def Page():
    with solara.Columns():
        with solara.Column(style={"width": "70%", "min-width": "650px"}):
            display(m)
        with solara.Column(style={"width": "30%", "min-width": "500px"}):
            SettingsTabs()
Page()
